In [1]:
import pandas as pd

# Read data from files 
file_train_labeled = 'C:/Users/Xin/Desktop/Github-repo/Natural_Language_Processing_Projects/Project_6_Movie_Review_Sentiment_Classification/Data/labeledTrainData.tsv'
file_train_unlabeled = 'C:/Users/Xin/Desktop/Github-repo/Natural_Language_Processing_Projects/Project_6_Movie_Review_Sentiment_Classification/Data/unlabeledTrainData.tsv'
file_test = 'C:/Users/Xin/Desktop/Github-repo/Natural_Language_Processing_Projects/Project_6_Movie_Review_Sentiment_Classification/Data/testData.tsv'
train = pd.read_csv( file_train_labeled, header=0, delimiter="\t", quoting=3 )
test = pd.read_csv( file_test, header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( file_train_unlabeled, header=0, delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print "Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size )

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [2]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [5]:
import nltk.data
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [9]:
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for review in train["review"]:
    sentences += review_to_sentences(review.decode("utf8"), tokenizer)

print "Parsing sentences from unlabeled set"
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review.decode("utf8"), tokenizer)


Parsing sentences from training set


C:\Users\Xin\Anaconda3\envs\ipykernel_py2\lib\site-packages\bs4\__init__.py:219: UserWarning: "." looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
C:\Users\Xin\Anaconda3\envs\ipykernel_py2\lib\site-packages\bs4\__init__.py:219: UserWarning: "..." looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
C:\Users\Xin\Anaconda3\envs\ipykernel_py2\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


C:\Users\Xin\Anaconda3\envs\ipykernel_py2\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\Xin\Anaconda3\envs\ipykernel_py2\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\Xin\Anaconda3\envs\ipykernel_py2\lib\site-packages\bs4\__init__.py:219: UserWarning: "... ..." looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  

In [12]:
print len(sentences)
print sentences[0]
print sentences[1]

795538
[u'with', u'all', u'this', u'stuff', u'going', u'down', u'at', u'the', u'moment', u'with', u'mj', u'i', u've', u'started', u'listening', u'to', u'his', u'music', u'watching', u'the', u'odd', u'documentary', u'here', u'and', u'there', u'watched', u'the', u'wiz', u'and', u'watched', u'moonwalker', u'again']
[u'maybe', u'i', u'just', u'want', u'to', u'get', u'a', u'certain', u'insight', u'into', u'this', u'guy', u'who', u'i', u'thought', u'was', u'really', u'cool', u'in', u'the', u'eighties', u'just', u'to', u'maybe', u'make', u'up', u'my', u'mind', u'whether', u'he', u'is', u'guilty', u'or', u'innocent']


In [14]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2016-10-22 20:43:54,994 : INFO : detected Windows; aliasing chunkize to chunkize_serial
2016-10-22 20:43:55,200 : INFO : Pattern library is not installed, lemmatization won't be available.
2016-10-22 20:43:55,217 : INFO : Could not import Theano, will use standard float for default ShardedCorpus dtype.
2016-10-22 20:43:55,585 : INFO : 'pattern' package not found; tag filters are not available for English
2016-10-22 20:43:55,721 : INFO : collecting all words and their counts
2016-10-22 20:43:55,723 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2016-10-22 20:43:55,822 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2016-10-22 20:43:55,904 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types


Training model...


2016-10-22 20:43:56,013 : INFO : PROGRESS: at sentence #30000, processed 671314 words, keeping 30034 word types
2016-10-22 20:43:56,095 : INFO : PROGRESS: at sentence #40000, processed 897814 words, keeping 34348 word types
2016-10-22 20:43:56,170 : INFO : PROGRESS: at sentence #50000, processed 1116962 words, keeping 37761 word types
2016-10-22 20:43:56,249 : INFO : PROGRESS: at sentence #60000, processed 1338403 words, keeping 40723 word types
2016-10-22 20:43:56,323 : INFO : PROGRESS: at sentence #70000, processed 1561579 words, keeping 43333 word types
2016-10-22 20:43:56,404 : INFO : PROGRESS: at sentence #80000, processed 1780886 words, keeping 45714 word types
2016-10-22 20:43:56,484 : INFO : PROGRESS: at sentence #90000, processed 2004995 words, keeping 48135 word types
2016-10-22 20:43:56,562 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2016-10-22 20:43:56,642 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 

In [15]:
model.doesnt_match("man woman child kitchen".split())

'kitchen'

In [16]:
model.doesnt_match("france england germany berlin".split())

'berlin'

In [17]:
model.doesnt_match("paris berlin london austria".split())

'paris'

In [18]:
model.most_similar("man")

[(u'woman', 0.6323821544647217),
 (u'lady', 0.6008610129356384),
 (u'lad', 0.5750747919082642),
 (u'monk', 0.566232442855835),
 (u'millionaire', 0.5291553735733032),
 (u'farmer', 0.5269814729690552),
 (u'chap', 0.5243933200836182),
 (u'soldier', 0.5153464674949646),
 (u'businessman', 0.5140076279640198),
 (u'men', 0.5137616395950317)]

In [19]:
model.most_similar("queen")

[(u'princess', 0.6909825801849365),
 (u'bride', 0.6298839449882507),
 (u'belle', 0.6224162578582764),
 (u'victoria', 0.5855280160903931),
 (u'stepmother', 0.5841797590255737),
 (u'maid', 0.5788012742996216),
 (u'maria', 0.5742377638816833),
 (u'countess', 0.565711498260498),
 (u'diana', 0.5642315149307251),
 (u'showgirl', 0.5621235370635986)]

In [20]:
model.most_similar("awful")

[(u'terrible', 0.7709794044494629),
 (u'atrocious', 0.7548998594284058),
 (u'horrible', 0.7409041523933411),
 (u'abysmal', 0.7279420495033264),
 (u'dreadful', 0.7190712690353394),
 (u'horrendous', 0.6869989633560181),
 (u'appalling', 0.6857318878173828),
 (u'horrid', 0.6836851835250854),
 (u'lousy', 0.6475538611412048),
 (u'amateurish', 0.6179372072219849)]

In [21]:
type(model.syn0)

numpy.ndarray

In [22]:
model.syn0.shape

(16490L, 300L)

In [23]:
model["flower"]

array([ 0.07611886,  0.07022257, -0.22491452,  0.04522414,  0.006568  ,
        0.04751567,  0.1393026 ,  0.00469399,  0.06333628, -0.01206978,
       -0.09721992, -0.18218544,  0.04218316,  0.13176009, -0.04569632,
        0.0386739 , -0.02008674,  0.00445849, -0.01756522, -0.1298802 ,
       -0.12459041, -0.01063596,  0.09662805,  0.01410297, -0.01573782,
       -0.01106897, -0.06547947, -0.0803933 , -0.04136204, -0.07225884,
       -0.0307914 ,  0.00168031, -0.01406776, -0.01447524, -0.0698203 ,
        0.04024277,  0.05038081, -0.0011916 ,  0.08872408, -0.00137332,
       -0.06026755,  0.00045231, -0.00643785, -0.1270586 ,  0.00595851,
        0.03259012, -0.04065161, -0.01163101, -0.000253  ,  0.01978572,
       -0.03932174,  0.05984711,  0.07226522,  0.02504538, -0.01278582,
        0.0217986 ,  0.03734752,  0.03872557, -0.04276543, -0.03074506,
       -0.10613407,  0.01799459,  0.10174102,  0.0411172 , -0.06514064,
        0.02205693,  0.04060026,  0.03889267,  0.04482412, -0.02

In [24]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print "Review %d of %d" % (counter, len(reviews))
       #
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs

In [25]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print "Creating average feature vecs for test reviews"
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 1900

In [26]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print "Fitting a random forest to labeled training data..."
forest = forest.fit( trainDataVecs, train["sentiment"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


In [27]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.syn0
num_clusters = word_vectors.shape[0] / 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print "Time taken for K Means clustering: ", elapsed, "seconds."

Time taken for K Means clustering:  540.781000137 seconds.


In [28]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.index2word, idx ))

In [29]:
# For the first 10 clusters
for cluster in xrange(0,10):
    #
    # Print the cluster number  
    print "\nCluster %d" % cluster
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        if( word_centroid_map.values()[i] == cluster ):
            words.append(word_centroid_map.keys()[i])
    print words


Cluster 0
[u'thora', u'cher', u'tilda', u'johansson', u'swinton', u'falco', u'gyllenhaal', u'scarlett', u'witherspoon', u'farrow', u'mia', u'cecilia', u'birch']

Cluster 1
[u'support', u'input', u'partnership', u'guidance', u'aid', u'assistance', u'interference', u'gifts']

Cluster 2
[u'destroys', u'oblivious', u'justifies', u'ignoring', u'undermines', u'contradicts', u'ignores']

Cluster 3
[u'speeds', u'burning', u'speeding']

Cluster 4
[u'matched', u'offset', u'heightened', u'reinforced', u'utilized', u'compensated', u'enhanced', u'rendered', u'provided', u'generated']

Cluster 5
[u'pasolini', u'plagiarism']

Cluster 6
[u'tribes', u'races', u'individuals', u'figures', u'units', u'families', u'churches', u'groups', u'generations', u'communities']

Cluster 7
[u'darryl']

Cluster 8
[u'welcomed', u'summoned', u'raised', u'harmed', u'formed', u'healed', u'defended', u'damaged', u'conquered', u'ruled', u'awakened', u'inventing', u'devastated', u'enslaved', u'controlled', u'examined', u'pr

In [30]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [31]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["review"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

In [32]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print "Fitting a random forest to labeled training data..."
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...
